# Cleaning Up Original Dataframe and Creating a States Dataframe


# Importing

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)

In [ ]:
def convert_percent_to_float(value):
    if isinstance(value, str) and '%' in value:
        return float(value.rstrip('%'))
    return value

def convert_comma_string_to_int(value):
    if isinstance(value, str) and ',' in value:
        return int(value.replace(',', ''))
    return value


df = pd.read_csv("US_Election_dataset_v1.csv")
df.head()


percent_cols = df[["Population with less than 9th grade education", "Population with 9th to 12th grade education, no diploma",
                   "High School graduate and equivalent", "Some College,No Degree", "Associates Degree", "Bachelors Degree", "Graduate or professional degree"]]

for col in percent_cols:
    df[col] = df[col].apply(convert_percent_to_float)

    df["Mean income (dollars)"] = df["Mean income (dollars)"].apply(convert_comma_string_to_int)
    df["Median income (dollars)"] = df["Median income (dollars)"].apply(convert_comma_string_to_int)

df.head()

# Finding and Replacing Missing Value

In [ ]:

#Find and Replace Missing Value
def find(df):
  val = input("What would you like to check your dataset for?")
  checking = [col for col in df.columns if (df[col] == val).any()]
  if checking:
    print(f"Columns containing '{val}': {checking}")
  else:
      print(f"No columns contain '{val}'.")
  return val

def replace(df, val):
  df.loc[df['Median income (dollars)'] == val, 'Median income (dollars)'] = 82787.41338582677
  return df


#Find and Replace
#Checking if there are any missing values
print(df.isnull().any())
print("Finding and Replacing Missing Value")
val = find(df)
df = replace(df, val)
find(df)

df["Voted Democrat"] = (df["2020 Democrat vote raw"] > df["2020 Republican vote raw"]).astype(int)

df.to_csv("US_Election_Counties_Dataset.csv")

Unnamed: 0                                                                   False
county                                                                       False
state                                                                        False
2020 Democrat vote raw                                                       False
2020 Democrat vote %                                                         False
2020 Republican vote raw                                                     False
2020 Republican vote %                                                       False
2020 other vote raw                                                          False
2020 other vote %                                                            False
Population with less than 9th grade education                                False
Population with 9th to 12th grade education, no diploma                      False
High School graduate and equivalent                                          False
Some

# Creating State Dataset

In [ ]:
def replace(df, val):
    df['Median income (dollars)'] = df['Median income (dollars)'].replace(val, 82787.41338582677)
    return df

def education_race_occupation(df, vote_columns, education_columns, race_columns, occupation_columns):
    #From Percents to Totals (per county)
    for perc_col in education_columns + race_columns + occupation_columns:
        df[perc_col] = df['Total Population'] * (df[perc_col] / 100)

    return df

def totals(df):
    df['Median income (dollars)'] = df['Median income (dollars)'].astype(float)
    #Grouping by State
    df = df.groupby("state").agg({
        # Votes
        "2020 Democrat vote raw": "sum",
        "2020 Republican vote raw": "sum",
        "2020 other vote raw": "sum",
        "Total Population": "sum",
        # Education
        "Population with less than 9th grade education": "sum",
        "Population with 9th to 12th grade education, no diploma": "sum",
        "High School graduate and equivalent": "sum",
        "Some College,No Degree": "sum",
        "Associates Degree": "sum",
        "Bachelors Degree": "sum",
        "Graduate or professional degree": "sum",
        # Metrics
        "Gini Index": "mean",
        "Mean income (dollars)": "mean",
        "Median income (dollars)": "mean",
        "Area in square Km": "sum",
        # Race
        "Hispanic or Latino percentage": "sum",
        "NH-White percentage": "sum",
        "NH-Black percentage": "sum",
        "NH-American Indian and Alaska Native percentage": "sum",
        "NH-Asian percentage": "sum",
        "NH-Native Hawaiian and Other Pacific Islander percentage": "sum",
        "NH-Some Other Race percentage": "sum",
        "NH-Two or More Races percentage": "sum",
        # Occupation
        "Percentage engaged in Management, business, science, and arts occupations": "sum",
        "Percentage engaged in Service Occupations": "sum",
        "Percentage engaged in Sales and Office": "sum",
        "Percentage engaged in Resources and Construction": "sum",
        "Percentage engaged in Transportation": "sum"
    }).reset_index()

    return df

def calc(df, vote_columns, education_columns, race_columns, occupation_columns):
    df['Total Votes'] = df['2020 Democrat vote raw'] + df['2020 Republican vote raw'] + df['2020 other vote raw']

    #Votes
    for pc_col, total_col in vote_columns:
        df[pc_col] = ((df[total_col] / df['Total Votes']) * 100).round(2)

    #Education
    for col in education_columns:
        df[col] = ((df[col] / df['Total Population']) * 100).round(2)

    #Metrics
    df['Density per square km'] = (df['Total Population'] / df['Area in square Km']).round(2)
    df['Gini Index'] = df['Gini Index'].round(4)
    df['Area in square Km'] = df['Area in square Km'].round(2)
    df['Mean income (dollars)'] = df['Mean income (dollars)'].round(2)
    df['Median income (dollars)'] = df['Median income (dollars)'].round(2)

    #Race
    for col in race_columns:
        df[col] = ((df[col] / df['Total Population']) * 100).round(2)

    #Occupation
    for col in occupation_columns:
        df[col] = ((df[col] / df['Total Population']) * 100).round(2)

    return df

#For State Abbreviations
def abbreviate(df):
    df['State Abbreviations'] = [
       'AL', 'AK', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DE', 'DC',
       'FL', 'GA', 'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY',
       'LA', 'ME', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT',
       'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC', 'ND', 'OH',
       'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT',
       'VT', 'VA', 'WA', 'WV', 'WI', 'WY'
    ]
    return df


#Column Names
vote_columns = [
        ('2020 Democrat vote %', '2020 Democrat vote raw'),
        ('2020 Republican vote %', '2020 Republican vote raw'),
        ('2020 other vote %', '2020 other vote raw')
    ]

education_columns = [
        'Population with less than 9th grade education',
        'Population with 9th to 12th grade education, no diploma',
        'High School graduate and equivalent',
        'Some College,No Degree',
        'Associates Degree',
        'Bachelors Degree',
        'Graduate or professional degree'
    ]

race_columns = [
        'Hispanic or Latino percentage',
        'NH-White percentage',
        'NH-Black percentage',
        'NH-American Indian and Alaska Native percentage',
        'NH-Asian percentage',
        'NH-Native Hawaiian and Other Pacific Islander percentage',
        'NH-Some Other Race percentage',
        'NH-Two or More Races percentage'
    ]

occupation_columns = [
        'Percentage engaged in Management, business, science, and arts occupations',
        'Percentage engaged in Service Occupations',
        'Percentage engaged in Sales and Office',
        'Percentage engaged in Resources and Construction',
        'Percentage engaged in Transportation'
    ]

#Run
df_state = df.copy()
df_state = education_race_occupation(df_state, vote_columns, education_columns, race_columns, occupation_columns)
df_state = totals(df_state)
df_state = calc(df_state, vote_columns, education_columns, race_columns, occupation_columns)
df_state = abbreviate(df_state)

df_state["Voted Democrat"] = (df_state["2020 Democrat vote raw"] > df_state["2020 Republican vote raw"]).astype(int)

df_state.to_csv("US_Election_States_Dataset.csv")
df_state.head()

,state,2020 Democrat vote raw,2020 Republican vote raw,2020 other vote raw,Total Population,Population with less than 9th grade education,"Population with 9th to 12th grade education, no diploma",High School graduate and equivalent,"Some College,No Degree",Associates Degree,Bachelors Degree,Graduate or professional degree,Gini Index,Mean income (dollars),Median income (dollars),Area in square Km,Hispanic or Latino percentage,NH-White percentage,NH-Black percentage,NH-American Indian and Alaska Native percentage,NH-Asian percentage,NH-Native Hawaiian and Other Pacific Islander percentage,NH-Some Other Race percentage,NH-Two or More Races percentage,"Percentage engaged in Management, business, science, and arts occupations",Percentage engaged in Service Occupations,Percentage engaged in Sales and Office,Percentage engaged in Resources and Construction,Percentage engaged in Transportation,Total Votes,2020 Democrat vote %,2020 Republican vote %,2020 other vote %,Density per square km,State Abbreviations,Voted Democrat
0,Alabama,849648,1441168,32488,5024279,3.70,8.55,30.32,21.20,8.92,16.75,10.58,0.4673,70621.25,51690.19,131185.10,5.26,63.12,25.64,0.46,1.51,0.05,0.29,3.67,36.03,15.93,20.67,9.76,17.60,2323304,36.57,62.03,1.40,38.30,AL,0
1,Alaska,153985,189925,15453,733391,2.23,4.39,29.03,24.90,8.89,19.13,11.41,0.4105,94283.27,76687.50,1479016.90,6.79,57.51,2.83,14.84,5.93,1.70,0.62,9.78,38.64,18.04,19.59,11.56,12.17,359363,42.85,52.85,4.30,0.50,AK,0
2,Arizona,1672143,1661686,53497,7151502,4.85,6.46,23.53,24.10,9.11,19.70,12.23,0.4495,77329.40,59725.53,294366.12,30.65,53.37,4.44,3.69,3.48,0.20,0.44,3.73,38.98,17.93,22.86,9.15,11.08,3387326,49.36,49.06,1.58,24.29,AR,1
3,Arkansas,423932,760647,34490,3011524,4.48,7.31,34.03,21.45,7.82,15.73,9.19,0.4618,67296.48,49542.33,134660.48,8.53,68.52,14.94,0.68,1.70,0.47,0.27,4.89,35.20,16.47,20.50,10.28,17.54,1219069,34.78,62.40,2.83,22.36,AZ,0
4,California,11109764,6005961,380181,39538223,8.80,6.94,20.58,20.23,7.99,21.88,13.58,0.4630,112918.57,82966.60,403673.31,39.40,34.69,5.36,0.39,15.12,0.35,0.57,4.12,41.15,17.70,19.99,9.04,12.13,17495906,63.50,34.33,2.17,97.95,CA,1
5,Colorado,1804352,1364607,87994,5773714,3.17,4.32,20.46,20.01,8.45,27.12,16.49,0.4490,94204.19,70542.53,268418.72,21.88,65.13,3.83,0.59,3.38,0.15,0.51,4.52,45.43,15.56,20.03,8.90,10.09,3256953,55.40,41.90,2.70,21.51,CO,1
6,Connecticut,1080831,714717,28309,3605944,3.82,4.50,25.14,16.06,7.44,23.44,19.62,0.4673,125886.38,90604.75,12178.67,17.29,63.21,10.01,0.18,4.73,0.03,0.75,3.82,46.66,16.36,19.87,7.07,10.03,1823857,59.26,39.19,1.55,296.09,CT,1
7,Delaware,296268,200603,7139,989948,3.16,5.58,29.69,18.69,8.31,19.91,14.60,0.4394,99445.00,76664.33,5046.71,10.54,58.57,21.51,0.26,4.29,0.03,0.47,4.34,41.14,17.59,20.52,8.88,11.87,504010,58.78,39.80,1.42,196.16,DE,1
8,DistrictofColumbia,39041,1725,915,689545,3.00,4.30,15.00,12.00,3.10,25.80,36.90,0.5171,150292.00,101722.00,158.32,11.26,37.96,40.91,0.19,4.81,0.05,0.54,4.28,69.47,11.93,12.96,2.30,3.34,41681,93.67,4.14,2.20,4355.39,DC,1
9,Florida,5297045,5668731,101680,21538187,4.39,6.38,27.63,19.13,10.07,20.29,12.13,0.4632,85043.99,61786.31,138963.82,26.45,51.54,14.52,0.19,2.92,0.05,0.64,3.68,37.84,18.75,23.44,9.35,10.62,11067456,47.86,51.22,0.92,154.99,FL,0
